In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src/")
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
checkpoint_path = "/mnt/SSD5/pholur/checkpoints/checkpoint.pt"
from dna2vec.trainer import ContrastiveTrainer
from dna2vec.model import model_from_config
# The path for the tokenizer isn't relative.
checkpoint = torch.load(checkpoint_path)
config = checkpoint["config"]
config.model_config.tokenizer_path = "/home/pholur/dna2vec/src/model/tokenizers/dna_tokenizer_10k.json"
encoder, pooling, tokenizer = model_from_config(config.model_config)

In [4]:
encoder

Encoder(
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (positional_embedding): SinusoidalPositionalEncoding()
  (embedding): Embedding(10004, 384)
  (trf_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=1536, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1536, out_features=384, bias=True)
        (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
)

In [5]:
from dna2vec.pinecone_store import PineconeStore
pinecone_store = PineconeStore(
    device = torch.device("cuda:4"),
    index_name = "dna-1-0504",
    metric = "cosine",
    model_params = {
        "tokenizer": tokenizer,
        "model": encoder,
        "pooling": pooling
    }
)

/mnt/SSD2/pholur/environments/dna2vec/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [15]:
query = "caccaacagtgtaaaagcgttcctatttctccacatcctctccagcatctgttgtttcctttttaatgatcgccat" #1195337
# query = "tgtgcccggccaatttttctctttaaaccCATTTACATTGTTTGAATTTTCTACCTGAGCAggaatttctttaataattaaaaaaaatcttccttaaCATTTAAGGGTCAGAGTCACCTATTACTGAACATGCCATTATTCCTGTAATCTGCATACCCCTTTGTTCATTCTGTGT" # 1838202

returned = pinecone_store.query([query])["matches"]
for sample in returned:
    returned = sample["metadata"]
    print(returned)

/mnt/SSD2/pholur/environments/dna2vec/lib/python3.11/site-packages/torch/nn/modules/transformer.py:562: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(


{'position': '634329', 'text': 'CACTCTGACCATCTGCATGCATAtctatctgtatttatttattttgggacag'}
{'position': '2185158', 'text': 'CACTCTTATATACAAGCCTCCTTATAAAAGGCCtagacttaaaaataataataatagcaaacacaT'}
{'position': '2474297', 'text': 'CACTCTCTTTCCCTGGAAGTTCTCAACCCTCATATTCCATATCCAACTATAATGT'}
{'position': '1279245', 'text': 'CACTCTTATCGGATGTAGTCATGTCACATAAGGGCTACTGAAGTGCAAGAGCAGTGATGCTGGTATacagtaataattgttttattattcatttctatT'}
{'position': '1427912', 'text': 'CACTCTGGGCATTACTGGGATGACAGACAGAAGAGCTAAAGGAGTGTTATTATTTAATCAAATGAAGCTATTG'}


In [17]:
with open("/home/pholur/dna2vec/tests/data/subsequences_sample_test.txt", "r") as f:
    lines = f.readlines()

distance = 0
from tqdm import tqdm
for line in tqdm(lines):
    text = line.split(" <> ")[0]
    index_scraped = int(line.split(" <> ")[1])
    returned_raw = pinecone_store.query([text], top_k=1)
    returned = int(returned_raw["matches"][0]["metadata"]["position"])
    distance += abs(index_scraped - returned)
    
print("Average distance: ", distance / len(lines))

100%|██████████| 1000/1000 [03:24<00:00,  4.90it/s]

Average distance:  1023152.533


In [18]:
import os
with open(os.path.join("/home/pholur/dna2vec/tests/data/NC_000002.12.txt"), "r") as f:
    sequence = f.read()
    print(len(sequence)/2)

1644887.0


In [ ]:
# Compute the edit distances
# 10-most similar